# Prelim ClinBall
Notes on gettings started with our data

In [2]:
import torch
import pandas as pd
import numpy as np

df = pd.read_csv('data/chr4_out.tsv', sep='\t', encoding = "UTF-8")
df = df.set_index(['chr','pos', 'ref', 'alt']) #index by identifier
#print(df)
#print(df.to_string()) ## will print the whole dataframe
#df.plot.hist()

print("shape before", df.shape)
df['MutationTaster_score'] = df['MutationTaster_score'].replace({'.': np.nan})
df = df.dropna()
print("shape after", df.shape)
df.head()

shape before (6016, 4)
shape after (4442, 4)


FATHMM_score MutationTaster_score integrated_fitCons_score  \
chr pos    ref alt                                                              
4   373410 A   T              1                 3.21       0.0130497586673599   
    373417 C   G       0.999992                 1.28       0.0126382143136347   
    373858 C   T       0.999887                 3.22      0.00763682056905044   
    500399 C   A       0.999112                  3.2        0.999689656609204   
    500405 C   A              1                 3.11        0.999999998480455   

                   GenoCanyon_score  
chr pos    ref alt                   
4   373410 A   T           0.562547  
    373417 C   G           0.562547  
    373858 C   T           0.732398  
    500399 C   A           0.542737  
    500405 C   A           0.542737

In [3]:
## Split into train/test

data_copy = df.copy()
df_train = data_copy.sample(frac=0.75, random_state=1)
df_test = data_copy.drop(df_train.index)

print ('Training data subset\n')
print (df_train.head(3))  # head default n=5, first 3 is enough
print ('\nTest data subset\n')
print (df_test.head(3))


## Get label: 
train_labels = df_train.pop('FATHMM_score')  # replace with clinvar when column is available
test_labels = df_test.pop('FATHMM_score')

Training data subset

                      FATHMM_score MutationTaster_score  \
chr pos       ref alt                                     
4   158682399 T   A              1                -3.87   
    113355493 G   A       0.999996                -0.09   
    523543    G   A       0.999326                 3.11   

                      integrated_fitCons_score GenoCanyon_score  
chr pos       ref alt                                            
4   158682399 T   A          0.999988431774354         0.706298  
    113355493 G   A          0.999998296604523         0.562547  
    523543    G   A          0.999135531316917         0.706548  

Test data subset

                   FATHMM_score MutationTaster_score integrated_fitCons_score  \
chr pos    ref alt                                                              
4   500405 C   G              1                 3.08        0.999999998480455   
    507558 G   A         0.8858                  0.9        0.987598326979718   
    50889

In [4]:
### NOT NEEDED, WAS EXAMPLE

import pandas as pd
import torch
import random

# creating dummy targets (float values)
targets_data = [random.random() for i in range(10)]

# creating DataFrame from targets_data
targets_df = pd.DataFrame(data=targets_data)
targets_df.columns = ['targets']

# creating tensor from targets_df 
torch_tensor = torch.tensor(targets_df['targets'].values)

# printing out result
print(torch_tensor)

tensor([0.0053, 0.2097, 0.4987, 0.3685, 0.8900, 0.6237, 0.3274, 0.4972, 0.5356,
        0.8351], dtype=torch.float64)


In [5]:
## Simple data import for train and testing datasets (if split), with pandas
#import pandas as pd
import numpy as np
from torch.utils import data as utils_data

train_tensor = torch.tensor(df_train.values.astype(np.float32))
train_labels_tensor = torch.tensor(train_labels.values.astype(np.float32))
print("Train tensor:\n", train_tensor)
                                     

#test = pd.read_csv('data/chr4_out.tsv')
test_tensor = torch.tensor(df_test.values.astype(np.float32))
test_labels_tensor = torch.tensor(test_labels.values.astype(np.float32))
print("Test tensor: \n", test_tensor)


### Create your data loaders:
train_X_y = utils_data.TensorDataset(train_tensor, train_labels_tensor)
training_loader = utils_data.DataLoader(train_X_y, batch_size=64, shuffle=True, num_workers=0)

# not sure that these need to be seperate
test_X_y = utils_data.TensorDataset(test_tensor, test_labels_tensor)
testing_loader = utils_data.DataLoader(test_X_y, batch_size=64, shuffle=True, num_workers=0)


### Check your loader
i1, l1 = next(iter(training_loader))
print(i1.shape)
print("Line 1", l1)
i2, l2 = next(iter(training_loader))
print(i2.shape)
print("Line 1", l2)  ## should be different

Train tensor:
 tensor([[-3.8700,  1.0000,  0.7063],
        [-0.0900,  1.0000,  0.5625],
        [ 3.1100,  0.9991,  0.7065],
        ...,
        [-0.9500,  1.0000,  0.7065],
        [-0.0500,  0.0063,  0.7063],
        [-0.4600,  0.9996,  0.6155]])
Test tensor: 
 tensor([[ 3.0800,  1.0000,  0.5427],
        [ 0.9000,  0.9876,  0.7065],
        [ 1.4900,  0.7329,  0.7065],
        ...,
        [-2.2700,  0.0047,  0.4871],
        [-2.8000,  0.0093,  0.4871],
        [-2.3100,  0.3826,  0.4871]])
torch.Size([64, 3])
Line 1 tensor([0.9893, 0.9453, 0.9841, 1.0000, 0.9999, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.9989, 0.6558, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.9997, 1.0000, 1.0000, 1.0000, 1.0000, 0.9943, 1.0000, 0.9998,
        1.0000, 1.0000, 0.5773, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9316,
        0.9999, 0.9999, 0.9999, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.9723, 1.0000, 1.0000, 1.0000, 0.9885, 0.9985,

In [6]:
#####  With data processed: get started with pytorch by straight up copying MNIST example + modifying

# Adapted from https://github.com/pytorch/examples/tree/master/mnist_hogwild 
from __future__ import print_function
import os

import torch
import torch.optim as optim
import torch.nn.functional as F  # functions are defined, such as relu, softmax, dropout 
import torch.nn as nn  # wherre the layers are defined
import torch.multiprocessing as mp
# from torchvision import datasets, transforms  #note for MK: I should know what is vision specific and address that

In [7]:
batch_size = 64   # help='input batch size for training (default: 64)'
test_batch_size = 1000   # help='input batch size for testing (default: 1000)'
epochs = 10       # help='number of epochs to train (default: 10)'
lr = 0.01         # help='learning rate (default: 0.01)'
momentum = 0.5    # help='SGD momentum (default: 0.5)'
seed = 1          # help='random seed (default: 1)'
log_interval = 100 # help='how many batches to wait before logging training status'
num_processes = 2 # help='how many training processes to use (default: 2)'
cuda = False      # help='enables CUDA training'

In [8]:
## CNN --> not working bc of tensor size incompatability
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  ## is in_channel/out_channel the number of nodes? pretty sure yes
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)  
        ## from this, the output chanels are 320, but you need to check the shape to know how to call the next layer
        self.fc1 = nn.Linear(320, 50)
        self.conv2_drop = nn.Dropout2d()
        self.fc1_drop = nn.Dropout()  #alt to F.dropout
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320) ## reshape to rearrange the data, MK does not understand quite how to get this value from their shapes
        x = F.relu(self.fc1(x))
        # x = F.dropout(x, training=self.training)  # possible to use dropout object instead
        x = self.fc1_drop(x)  # p default 0.5, inplate default False
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  #log prob of each of the 10 classes/digits. not compatible with cross-entropy (log_softmax and nll)
    
    # note that if you want to use cross-entropy, use that instead of the nll below (and remove the above usage of log_softmax, instead forward would just return X)

In [14]:
class mlp(nn.Module):
    def __init__(self, dim=200, input_size=100, output_size=10, num_layers=4):
        super(mlp_generator, self).__init__()

        num_layers = num_layers + 2  # add 2 for the 1st and the last layers
        modules = []
        for layer in range(num_layers): 
            in_size = input_size if layer == 0 else dim
            out_size = output_size if layer == num_layers - 1 else dim
            if layer < num_layers - 1:
                modules.append(
                    nn.Sequential(
                        nn.Linear(in_size, out_size),
                        nn.ReLU(True),
                    )
                )
            else:
                modules.append(
                    nn.Sequential(
                        nn.Linear(in_size, out_size),
                        nn.Tanh()
                    )
                )
        self.layers = nn.Sequential(*modules)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x
    
## used: netG = mlp_generator(args.gen_dim, args.latent_dim, args.max_seq_len*vocab_size, args.gen_layers).to(device)

In [9]:
torch.manual_seed(seed)
#use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cpu")  # torch.device("cuda" if use_cuda else "cpu") # 
dataloader_kwargs = {}

In [ ]:

## don't run this, see what breaks
torch.manual_seed(seed)
#use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cpu")  # torch.device("cuda" if use_cuda else "cpu") # 
dataloader_kwargs = {}# {'pin_memory': True} if use_cuda else {}  #maybe buggy, not clear if you need this 


##### These are the loaders for MNIST, we need to make our own
train_loader_MNIST = torch.utils.data.DataLoader( 
    datasets.MNIST('./data', train=True, download=False,  # set download to True for first run
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, num_workers=1, **dataloader_kwargs)


test_loader_MNIST = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, download=False, # set download to True for first run
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, num_workers=1, **dataloader_kwargs)



In [10]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # type of optimization algo

model.conv1.weight   ## play with the model's methods and attributes
## use tensor.shape is to check torch.Size 
print(model.conv1.weight.shape)

torch.Size([10, 1, 5, 5])


In [11]:
# Training
for epoch in range(1, epochs + 1):
    print("epoch, ", epoch)
    model.train()
    pid = os.getpid()

    for batch_idx, (data, target) in enumerate(training_loader):
        optimizer.zero_grad()  # set the gradients to zero to start
        output = model(data.to(device))
        
        # loss is a tensor that is attached to the graph of computation, this is the tape that joseph talked about
        loss = F.nll_loss(output, target.to(device))  #negative log-liklihood loss function (applied to one forward iteration)
        # loss *= 0.1  ## Note: this would actually affect the gradient 
        val = loss.item() * 0.1  ## note that this would store the value, but not change it. Can look at detach for remove. 
        loss.backward() # calculate the gradients to see what to "adjust"
        optimizer.step()  # use the gradients to update the weights
        
        if batch_idx % log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(training_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

epoch,  1


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 10 1 5 5, but got 2-dimensional input of size [64, 3] instead

In [ ]:
# Test
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():  ## important not to add these gradients to the loss above (will affect model if you do)
    for data, target in test_loader:
        output = model(data.to(device))
        # print(output)
        test_loss += F.nll_loss(output, target.to(device), reduction='sum').item() # sum up batch loss
        # print(test_loss)
        pred = output.max(1)[1] # get the index of the max log-probability  
        correct += pred.eq(target.to(device)).sum().item()  ## this is part of MNIST dataset, ours will be different. 

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

print(model)